In [31]:
with open('D:/Users/ihera/OneDrive/Escritorio/test.txt', 'r') as archivo:
    contenido = archivo.read()

print(contenido)

Oh, goddamn
My pain fits in the palm of your freezing hand
Taking mine, but it's been promised to another
Oh, I can't
Stop you putting roots in my dreamland
My house of stone, your ivy grows
And now I'm covered in you


In [32]:
import random
import time
import numpy as np
import heapq
import json

In [44]:
# Esquema comunicación

# Fuente de información
class FuenteInformacion:
    def __init__(self, archivo):
        self.archivo = archivo

    def generar_datos(self):
        return self.archivo
    
class Codificar:
    # Huffman
    def codigoHuffman(self, lista):
        freq = {}
        for elemento in lista:
            if elemento in freq:
                freq[elemento] += 1
            else:
                freq[elemento] = 1

         # Crear un nodo hoja para cada símbolo, asociando un peso según su frecuencia de aparición e insertarlo en la lista ordenada ascendentemente.
        nodos = [Nodo(simbolo, frecuencia) for simbolo, frecuencia in freq.items()]
        heapq.heapify(nodos)

        # Mientras haya mas de un nodo en la lista.
        while len(nodos) > 1:
            # Eliminar los dos nodos con menos probabilidad de la lista
            nodo_izquierdo = heapq.heappop(nodos)
            nodo_derecho = heapq.heappop(nodos)
            # Asignar valor direccional
            nodo_izquierdo.huff = 0
            nodo_derecho.huff = 1
            # Crear un nuevo nodo interno que enlace a los nodos anteriores, asignándoles como peso la suma de los pesos de los nodos hijos.
            nuevo_nodo = Nodo(None, nodo_izquierdo.freq + nodo_derecho.freq)
            nuevo_nodo.left = nodo_izquierdo
            nuevo_nodo.right = nodo_derecho
            heapq.heappush(nodos, nuevo_nodo)

        return nodos[0]
        
    def guardarNodos(self, nodo, val='', huffDict=None):
        if huffDict is None:
            huffDict = {}

        nuevoVal = val + str(nodo.huff)

        if nodo.left:
            self.guardarNodos(nodo.left, nuevoVal, huffDict)
        if nodo.right:
            self.guardarNodos(nodo.right, nuevoVal, huffDict)

        if not nodo.left and not nodo.right:
            huffDict[nodo.sim] = nuevoVal
            
        return huffDict

    # Shannon-Fano
    def shannon_fano(self, lista):
        # Calcula las probabilidades de cada símbolo en la lista
        probabilidad = {}
        for elemento in lista:
            if elemento in probabilidad:
                probabilidad[elemento] += 1
            else:
                probabilidad[elemento] = 1

        # Ordena los símbolos por probabilidad en orden descendente
        simbolos_ordenados = sorted(probabilidad.items(), key=lambda x: x[1], reverse=True)

        # Función recursiva para dividir y asignar '0' y '1'
        def dividir_asignar(simbolos, prefijo=''):
            if len(simbolos) == 1:
                return {simbolos[0][0]: prefijo}
            elif len(simbolos) == 2:
                return {simbolos[0][0]: prefijo + '0', simbolos[1][0]: prefijo + '1'}
            else:
                total_probabilidad = sum(prob for _, prob in simbolos)
                mitad_probabilidad = 0
                split_index = -1

                for i, (_, prob) in enumerate(simbolos):
                    if mitad_probabilidad <= total_probabilidad / 2:
                        mitad_probabilidad += prob
                    else:
                        split_index = i
                        break

                grupo1 = simbolos[:split_index]
                grupo2 = simbolos[split_index:]

                asignaciones_grupo1 = dividir_asignar(grupo1, prefijo + '0')
                asignaciones_grupo2 = dividir_asignar(grupo2, prefijo + '1')

                asignaciones = {}
                asignaciones.update(asignaciones_grupo1)
                asignaciones.update(asignaciones_grupo2)

                return asignaciones

        shannon_fano_dict = dividir_asignar(simbolos_ordenados)

        return shannon_fano_dict

    # Codificar
    def codificar(self, cadena, dict):
        codigo = []
        for caracter in cadena:
            codigo.append(dict[caracter])
        return codigo
    
class Transmisor:
    def __init__(self, fuente, canal, canales):
        self.fuente = fuente
        self.canal = canal
        self.canales = canales
        self.codificador = Codificar()
    
    def transmitir(self):
        # Encode
        datos = self.fuente.generar_datos()
        datos_bytes = datos.encode('utf-8')
        datos_bin = ''.join(format(byte, '08b') for byte in datos_bytes)
        paquetes = [datos_bin[i:i+8] for i in range(0, len(datos_bin), 8)]

        while True:
            choice = input("Escoja un esquema de codificación (Huffman o Shannon-Fano): ").strip().lower()
            if choice == "huffman":
                arbol = self.codificador.codigoHuffman(paquetes)
                diccionario = self.codificador.guardarNodos(arbol)
                codigo = self.codificador.codificar(paquetes, diccionario)
                break
            elif choice == "shannon-fano":
                diccionario = self.codificador.shannon_fano(paquetes)
                codigo = self.codificador.codificar(paquetes, diccionario)
                break
            else:
                print("Opción invalida.")
        # Encode
        print(f'Codificando con: {choice.capitalize()}')
        # Transmitir
        canal_actual = random.choice(self.canales)
        paquete_actual = 0
        print(f'Enviando por el canal: {canal_actual}')
        # Transmitir el diccionario primero como un paquete
        diccionario_str = json.dumps(diccionario)
        diccionario_packet = Paquete(diccionario_str, paquete_actual)
        if not self.canal.enviar(diccionario_packet, canal_actual):
            print("Diccionario perdido. Cambiando a canal de respaldo...")
            index = self.canales.index(canal_actual)
            canal_actual = self.canales[(index + 1) % len(self.canales)]
            print(f'Enviando por el canal: {canal_actual}')

        while paquete_actual < len(paquetes):
            paquete = Paquete(codigo[paquete_actual], paquete_actual)
            if not self.canal.enviar(paquete, canal_actual):
                index = self.canales.index(canal_actual)
                canal_actual = self.canales[(index + 1) % len(self.canales)]
                print(f'Paquete perdido. Cambiando a canal {canal_actual}...')
            else:
                paquete_actual += 1

class CanalComunicacion:
    def __init__(self, velocidad, receptor):
        self.velocidad = velocidad
        self.receptor = receptor
    
    def enviar(self, paquete, num_canal):
        # print(f'Transmitiendo por el canal: {num_canal}')
        time.sleep(self.velocidad)
        if random.random() > 0.09:
            if self.receptor.recibir(paquete):
                return True
        return False

class Receptor:
    def __init__(self):
        self.paquetes = []
        self.coding_dict = None
        self.paquete_esperado = 0

    def recibir(self, paquete):
        if paquete.num_paquete == self.paquete_esperado:
            if self.coding_dict is None:
                # El primer paquete contiene el diccionario
                try:
                    coding_dict_str = paquete.datos
                    self.coding_dict = json.loads(coding_dict_str)
                except json.JSONDecodeError:
                    pass
            else:
                # Paquetes
                self.paquetes.append(paquete.datos)
                self.paquete_esperado += 1
                return True
        return False

    def decodificar_mensaje(self):
        mensaje_binario = ''
        simbolo_actual = ''

        for bit in self.paquetes:
            simbolo_actual += bit
            for simbolo, codificion in self.coding_dict.items():
                if codificion == simbolo_actual:
                    mensaje_binario += simbolo
                    simbolo_actual = ''
                    break

        mensaje_bytes = [int(mensaje_binario[i:i+8], 2) for i in range(0, len(mensaje_binario), 8)]
        mensaje_decodificado = bytes(mensaje_bytes).decode('utf-8')
        return mensaje_decodificado

class Paquete:
    def __init__(self, datos, num_paquete):
        self.datos = datos
        self.num_paquete = num_paquete

class Nodo:
    def __init__(self, sim, freq, left=None, right=None):
        self.freq = freq
        self.sim = sim
        self.left = left
        self.right = right
        self.huff = ''

    #  Determina qué elemento tiene la mayor prioridad y, por lo tanto, se extrae primero
    def __lt__(self, siguiente):
        return self.freq < siguiente.freq

In [45]:
contenido_texto = "Este es un ejemplo de contenido de archivo de texto."
canales = [1, 2, 3, 4, 5]

fuente_info = FuenteInformacion(contenido)
receptor = Receptor()
canal = CanalComunicacion(0.1, receptor)
transmisor = Transmisor(fuente_info, canal, canales)

transmisor.transmitir()

# Al final de la transmisión, decodificar el mensaje
mensaje_decodificado = receptor.decodificar_mensaje()
print(f"Mensaje decodificado: {mensaje_decodificado}")

Codificando con: Shannon-fano
Enviando por el canal: 4
Diccionario perdido. Cambiando a canal de respaldo...
Enviando por el canal: 5
Paquete perdido. Cambiando a canal 1...
Paquete perdido. Cambiando a canal 2...
Paquete perdido. Cambiando a canal 3...
Paquete perdido. Cambiando a canal 4...
Paquete perdido. Cambiando a canal 5...
Paquete perdido. Cambiando a canal 1...
Paquete perdido. Cambiando a canal 2...
Paquete perdido. Cambiando a canal 3...
Paquete perdido. Cambiando a canal 4...
Paquete perdido. Cambiando a canal 5...
Paquete perdido. Cambiando a canal 1...
Paquete perdido. Cambiando a canal 2...
Paquete perdido. Cambiando a canal 3...
Paquete perdido. Cambiando a canal 4...
Paquete perdido. Cambiando a canal 5...
Paquete perdido. Cambiando a canal 1...
Paquete perdido. Cambiando a canal 2...
Mensaje decodificado: Oh, goddamn
My pain fits in the palm of your freezing hand
Taking mine, but it's been promised to another
Oh, I can't
Stop you putting roots in my dreamland
My hous